<a href="https://colab.research.google.com/github/nabilaaaa24/data_mining/blob/main/Praktikum_Modul_4_Kegiatan-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import os
import torch
from tqdm.notebook import tqdm
import pickle
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv('/content/drive/MyDrive/dataset_komentar_instagram_cyberbullying.csv')
df

,Id,Sentiment,Instagram Comment Text
0,1,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,2,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,3,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,4,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,5,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...,...
395,396,positive,Bangga sama suami yg selalu ingat istri disela...
396,397,positive,Apaoun pekerjaannya yg penting halal u tuk men...
397,398,positive,Gojek itu mayoritas pegangguran yang lama gak ...
398,399,positive,<USERNAME> aslinya cantik dan ayu loh mbak kr...


## Data Cleaning

In [4]:
def casefolding(text):
  text = text.lower()                                   # mengubah teks menjadi lower case
  text = re.sub(r'https?://\S+|www\.\s+', '', text)     # menghapus URL
  text = re.sub(r'[-+]?[0-9]+', '', text)               # menghapus angka
  text = re.sub(r'[^\w\s]', '', text)                   # menghapus karakter tanda baca
  text = text.strip()
  return text

In [5]:
raw_sample = df['Instagram Comment Text'].iloc[3]
case_folding = casefolding(raw_sample)

In [6]:
print('Raw data     : ', raw_sample)
print('Case folding : ', case_folding)

Raw data     :  Intinya kalau kesel dengan ATT nya, gausah ke anaknya juga. Kasian buat perkembangan psikis anak kedepannya. Itu orang bener bener tolol, skrg seandainya dia punya anak, terus anaknya dikatain sama orang yang benci sama dia, gimana perasaan dia ? Benci sama seseorang boleh, tapi harus tau batesnya ?? toh namanya manusia, gaakan semua jadi penyuka, pasti ada haters ??
Case folding :  intinya kalau kesel dengan att nya gausah ke anaknya juga kasian buat perkembangan psikis anak kedepannya itu orang bener bener tolol skrg seandainya dia punya anak terus anaknya dikatain sama orang yang benci sama dia gimana perasaan dia  benci sama seseorang boleh tapi harus tau batesnya  toh namanya manusia gaakan semua jadi penyuka pasti ada haters


## Stopword Removal

In [7]:
!wget https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv

--2024-05-06 03:42:07--  https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80969 (79K) [text/plain]
Saving to: ‘key_norm.csv’

key_norm.csv        100%[===================>]  79.07K  --.-KB/s    in 0.001s  

2024-05-06 03:42:07 (56.7 MB/s) - ‘key_norm.csv’ saved [80969/80969]



In [8]:
key_norm = pd.read_csv('key_norm.csv')
print(key_norm)

key_norm.shape

       _id      singkat        hasil
0        1         abis        habis
1        2       accent      tekanan
2        3       accept       terima
3        4     accident   kecelakaan
4        5  achievement     prestasi
...    ...          ...          ...
3715  3716        ywdah     ya sudah
3716  3717         ywdh     ya sudah
3717  3718       ywdhlh  ya sudahlah
3718  3719         ywis     ya sudah
3719  3720           rp       rupiah

[3720 rows x 3 columns]


(3720, 3)

In [9]:
def text_normalize(text):
  text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
  text = str.lower(text)
  return text

In [10]:
stopwords_ind = stopwords.words('indonesian')
len(stopwords_ind)

758

In [11]:
# lihat daftar stopword yang disediakan NLTK
stopwords_ind[:20]

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara']

In [12]:
more_stopword = ['geblek', 'tolol', 'bangsat', 'username', 'nya']
stopwords_ind = stopwords_ind + more_stopword

def remove_stop_words(text):
  clean_words = []
  text = text.split()
  for word in text:
    if word not in stopwords_ind:
      clean_words.append(word)
  return " ".join(clean_words)

In [13]:
raw_sample = df['Instagram Comment Text'].iloc[3]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data         : ', raw_sample)
print('Case folding     : ', case_folding)
print('Stopword removal : ', stopword_removal)

Raw data         :  Intinya kalau kesel dengan ATT nya, gausah ke anaknya juga. Kasian buat perkembangan psikis anak kedepannya. Itu orang bener bener tolol, skrg seandainya dia punya anak, terus anaknya dikatain sama orang yang benci sama dia, gimana perasaan dia ? Benci sama seseorang boleh, tapi harus tau batesnya ?? toh namanya manusia, gaakan semua jadi penyuka, pasti ada haters ??
Case folding     :  intinya kalau kesel dengan att nya gausah ke anaknya juga kasian buat perkembangan psikis anak kedepannya itu orang bener bener tolol skrg seandainya dia punya anak terus anaknya dikatain sama orang yang benci sama dia gimana perasaan dia  benci sama seseorang boleh tapi harus tau batesnya  toh namanya manusia gaakan semua jadi penyuka pasti ada haters
Stopword removal :  intinya kesel att gausah anaknya kasian perkembangan psikis anak kedepannya orang bener bener skrg seandainya anak anaknya dikatain orang benci gimana perasaan benci tau batesnya namanya manusia gaakan penyuka hater

## Stemming

In [14]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#buat fungsi untuk langkah stemming bahasa indonesia
def stemming(text):
  text = stemmer.stem(text)
  return text

In [15]:
aw_sample = df['Instagram Comment Text'].iloc[3]
case_folding = casefolding(raw_sample)
stopword_removal =  remove_stop_words(case_folding)
text_stemming = stemming(stopword_removal)

print('Raw data:', raw_sample)
print('Case folding:', case_folding)
print('Stopword removal:', stopword_removal)
print('Stemming:', text_stemming)

Raw data: Intinya kalau kesel dengan ATT nya, gausah ke anaknya juga. Kasian buat perkembangan psikis anak kedepannya. Itu orang bener bener tolol, skrg seandainya dia punya anak, terus anaknya dikatain sama orang yang benci sama dia, gimana perasaan dia ? Benci sama seseorang boleh, tapi harus tau batesnya ?? toh namanya manusia, gaakan semua jadi penyuka, pasti ada haters ??
Case folding: intinya kalau kesel dengan att nya gausah ke anaknya juga kasian buat perkembangan psikis anak kedepannya itu orang bener bener tolol skrg seandainya dia punya anak terus anaknya dikatain sama orang yang benci sama dia gimana perasaan dia  benci sama seseorang boleh tapi harus tau batesnya  toh namanya manusia gaakan semua jadi penyuka pasti ada haters
Stopword removal: intinya kesel att gausah anaknya kasian perkembangan psikis anak kedepannya orang bener bener skrg seandainya anak anaknya dikatain orang benci gimana perasaan benci tau batesnya namanya manusia gaakan penyuka haters
Stemming: inti k

In [16]:
def text_preprocessing_process(text):
  text = casefolding(text)
  text = stemming(text)
  text = remove_stop_words(text)
  return text

In [17]:
df['clean_teks'] = df['Instagram Comment Text'].apply(text_preprocessing_process)

In [18]:
df

,Id,Sentiment,Instagram Comment Text,clean_teks
0,1,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,gak hubung gugur dgn pake hijab syar yg lo bil...
1,2,negative,Geblek lo tata...cowo bgt dibela2in balikan......,lo tatacowo bgt bain balikanhadewwntar tinggal...
2,3,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn mewek skr lengket lg duhhh labil bgt sih ...
3,4,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ...",inti kesel att gausah anak kasi kembang psikis...
4,5,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",hadewwwww permpuan lgsakit jiwaknp yg jd peran...
...,...,...,...,...
395,396,positive,Bangga sama suami yg selalu ingat istri disela...,bangga suami yg istri profesi kau juara sehat ...
396,397,positive,Apaoun pekerjaannya yg penting halal u tuk men...,apaoun kerja yg halal u tuk nafkah keluarga se...
397,398,positive,Gojek itu mayoritas pegangguran yang lama gak ...,gojek mayoritas pegangguran gak kerja trus put...
398,399,positive,<USERNAME> aslinya cantik dan ayu loh mbak kr...,asli cantik ayu loh mbak krn temu langsung sem...


In [19]:
df.to_csv('clean_data.csv')

# Pelatihan Model Klasifikasi

In [20]:
tf_idf = TfidfVectorizer()
X = tf_idf.fit_transform(df['clean_teks']).toarray()
y = df['Sentiment']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
svm_model = SVC(kernel='linear', random_state=42)

nb_model = GaussianNB()

In [23]:
svm_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [24]:
nb_model.fit(X_train, y_train)

GaussianNB()

# Evaluasi Hasil Klasifikasi

In [25]:
y_pred_svm = svm_model.predict(X_test)
y_pred_nb = nb_model.predict(X_test)

In [26]:
print("Classification Report Support Vector Mechine model:")
print(classification_report(y_test, y_pred_svm))

Classification Report Support Vector Mechine model:
              precision    recall  f1-score   support

    negative       0.97      0.84      0.90        44
    positive       0.83      0.97      0.90        36

    accuracy                           0.90        80
   macro avg       0.90      0.91      0.90        80
weighted avg       0.91      0.90      0.90        80



In [27]:
print("F1 Score:", f1_score(y_test, y_pred_svm, average="macro"))
print("Precision Score:", precision_score(y_test, y_pred_svm, average="macro"))
print("Recall Score:", recall_score(y_test, y_pred_svm, average="macro"))
print("Accuracy Score:", accuracy_score(y_test, y_pred_svm))

F1 Score: 0.8999374609130707
Precision Score: 0.9035087719298246
Recall Score: 0.9065656565656566
Accuracy Score: 0.9


In [28]:
print("Classification Report Naive Bayes model:")
print(classification_report(y_test, y_pred_nb))

Classification Report Naive Bayes model:
              precision    recall  f1-score   support

    negative       0.80      0.84      0.82        44
    positive       0.79      0.75      0.77        36

    accuracy                           0.80        80
   macro avg       0.80      0.80      0.80        80
weighted avg       0.80      0.80      0.80        80



In [29]:
print("F1 Score:", f1_score(y_test, y_pred_nb, average="macro"))
print("Precision Score:", precision_score(y_test, y_pred_nb, average="macro"))
print("Recall Score:", recall_score(y_test, y_pred_nb, average="macro"))
print("Accuracy Score:", accuracy_score(y_test, y_pred_nb))

F1 Score: 0.7968253968253969
Precision Score: 0.79923273657289
Recall Score: 0.7954545454545454
Accuracy Score: 0.8
